In [1]:
import os
import gc
os.chdir('../../')

In [2]:
import pandas as pd
import numpy as np
from week_1.notebooks.map_jobs_to_categories import map_jobs_to_categories
from global_code.util import reduce_mem_usage
import json

/Users/marcus.silva/Code/modern-ml/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Read the parquet file located at `./week_1/data/processed/full_churn_data_with_target.parquet`

In [3]:
df = pd.read_parquet('./week_1/data/processed/full_churn_data_with_target.parquet')
df.reset_index(drop=False, inplace=True) 
df.sort_values(by=['customer_id', 'date'], ascending=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5286530 entries, 0 to 5286529
Data columns (total 31 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   Id                        int64         
 1   customer_id               int64         
 2   interest_rate             float64       
 3   name                      object        
 4   country                   object        
 5   date_of_birth             object        
 6   address                   object        
 7   date                      datetime64[ns]
 8   atm_transfer_in           int64         
 9   atm_transfer_out          int64         
 10  bank_transfer_in          int64         
 11  bank_transfer_out         int64         
 12  crypto_in                 int64         
 13  crypto_out                int64         
 14  bank_transfer_in_volume   float64       
 15  bank_transfer_out_volume  float64       
 16  crypto_in_volume          float64       
 17  crypto_o

In [4]:
df = reduce_mem_usage(df)

Memory usage of dataframe is 1144.45 MB
Memory usage after optimization is: 675.58 MB
Decreased by 41.0%


### Feature Engineering

In [5]:
# Touchpoints nested list to individual count columns

# 1. Explode the touchpoints list column to transform each channel into its own row
exploded_df = df.explode('touchpoints')

# 2. Count the occurrences of each channel per original row
counts = (exploded_df.groupby([exploded_df.index, 'touchpoints'])
                     .size()
                     .unstack(fill_value=0))

exploded_df = None

# 3. Ensure all possible columns exist (email, appointment, phone, whatsapp)
for col in ['email', 'appointment', 'phone', 'whatsapp']:
    if col not in counts.columns:
        counts[col] = 0

# 4. Merge these counts back into the original DataFrame
df = df.join(counts, how='left').fillna(0)
counts = None

# 5. Convert counts to integer type
df[['email', 'appointment', 'phone', 'whatsapp']] = (
    df[['email', 'appointment', 'phone', 'whatsapp']].astype(int)
)

gc.collect()

0

In [6]:
df['days_between'] = df.groupby('customer_id')['date'].diff().dt.days.fillna(0)
df['customer_age'] = (df['date'] - pd.to_datetime(df['date_of_birth'])).dt.days / 365.25
df['from_competitor'] = df['from_competitor'].astype(int)
df['churn_due_to_fraud'] = df['churn_due_to_fraud'].astype(int)

In [7]:
# Jobs to categories
map_jobs_to_categories(df)

In [8]:
def add_window_features(df, window_size=''):
    print('Adding window features for', window_size, '...')
    # Define the aggregation dictionary for rolling operations
    agg_dict = {
        'date': 'count',
        'days_between': ['sum', 'mean', 'std', 'max', 'min'],
        'bank_transfer_in': 'mean',
        'bank_transfer_out': 'mean',
        'crypto_in': 'mean',
        'crypto_out': 'mean',
        'bank_transfer_in_volume': ['mean', 'sum'],
        'bank_transfer_out_volume': ['mean', 'sum'],
        'crypto_in_volume': ['mean', 'sum'],
        'crypto_out_volume': ['mean', 'sum']
    }
    
    # Perform the groupby and rolling aggregation
    result = (
        df.groupby('customer_id')
          .rolling(window_size, on='date')
          .agg(agg_dict)
    )
    
    # Flatten the MultiIndex columns and rename them
    result.columns = [
        f"prior_{window_size}_{stat}_{col}" if stat != '' else f"prior_{window_size}_{col}"
        for col, stat in result.columns
    ]
    

    # Reduce memory
    result = reduce_mem_usage(result)

    # Reset index if needed (depending on your use case)
    result = result.reset_index()
    
    # Merge back with the original DataFrame if required
    df = df.merge(result, on=['customer_id', 'date'], how='left')

    # Interations features
    df[f'prior_{window_size}_bank_volume'] = df[f'prior_{window_size}_sum_bank_transfer_in_volume']- df[f'prior_{window_size}_sum_bank_transfer_out_volume']
    df[f'prior_{window_size}_crypto_volume'] = df[f'prior_{window_size}_sum_crypto_in_volume']  - df[f'prior_{window_size}_sum_crypto_out_volume']
    
    df[f'prior_{window_size}_bank_balance'] = (
        df[f'prior_{window_size}_bank_volume'] + df[f'prior_{window_size}_crypto_volume']
    )

    df[f'prior_{window_size}_mean_balance'] = (
        (df[f'prior_{window_size}_mean_bank_transfer_in_volume'] - df[f'prior_{window_size}_mean_bank_transfer_out_volume'])
        + (df[f'prior_{window_size}_mean_crypto_in_volume'] - df[f'prior_{window_size}_mean_crypto_out_volume'])
    )
    return df

# Last 7 days
df = add_window_features(df, '7D')

# Last 10 days
df = add_window_features(df, '10D')

# Last 90 days
df = add_window_features(df, '90D')

# Last 180 days
df = add_window_features(df, '180D')

# Last 365 days
df = add_window_features(df, '365D')

# Last 365 days
df = add_window_features(df, '450D')

Adding window features for 7D ...
Memory usage of dataframe is 757.18 MB
Memory usage after optimization is: 253.02 MB
Decreased by 66.6%
Adding window features for 10D ...
Memory usage of dataframe is 757.18 MB
Memory usage after optimization is: 253.02 MB
Decreased by 66.6%
Adding window features for 90D ...
Memory usage of dataframe is 757.18 MB
Memory usage after optimization is: 253.02 MB
Decreased by 66.6%
Adding window features for 180D ...
Memory usage of dataframe is 757.18 MB
Memory usage after optimization is: 253.02 MB
Decreased by 66.6%
Adding window features for 365D ...
Memory usage of dataframe is 757.18 MB
Memory usage after optimization is: 253.02 MB
Decreased by 66.6%
Adding window features for 450D ...
Memory usage of dataframe is 757.18 MB
Memory usage after optimization is: 253.02 MB
Decreased by 66.6%


In [9]:
# Lifetime Window features
print('Lifetime Window features...')
df['prior_emails'] = df.groupby('customer_id')['email'].cumsum().values
df['prior_appointments'] = df.groupby('customer_id')['appointment'].cumsum().values
df['prior_phones'] = df.groupby('customer_id')['phone'].cumsum().values
df['prior_whatsapps'] = df.groupby('customer_id')['whatsapp'].cumsum().values

df['prior_touchpoints'] = df['prior_emails'] + df['prior_appointments'] + df['prior_phones'] + df['prior_whatsapps']
df['prior_count'] = df.groupby('customer_id').cumcount() + 1

df['prior_sum_days_between'] = df.groupby('customer_id')['days_between'].expanding().sum().reset_index(level=0, drop=True)
df['prior_std_days_between'] = df.groupby('customer_id')['days_between'].expanding().std().reset_index(level=0, drop=True)
df['prior_mean_days_between'] = df.groupby('customer_id')['days_between'].expanding().mean().reset_index(level=0, drop=True)
df['prior_min_days_between'] = df.groupby('customer_id')['days_between'].expanding().min().reset_index(level=0, drop=True)
df['prior_max_days_between'] = df.groupby('customer_id')['days_between'].expanding().max().reset_index(level=0, drop=True)

# Transfers and volumes
df['prior_mean_bank_transfer_in'] = df.groupby('customer_id')['bank_transfer_in'].expanding().mean().reset_index(level=0, drop=True)
df['prior_mean_bank_transfer_out'] = df.groupby('customer_id')['bank_transfer_out'].expanding().mean().reset_index(level=0, drop=True)
df['prior_mean_crypto_in'] = df.groupby('customer_id')['crypto_in'].expanding().mean().reset_index(level=0, drop=True)
df['prior_mean_crypto_out'] = df.groupby('customer_id')['crypto_out'].expanding().mean().reset_index(level=0, drop=True)
df['prior_mean_bank_transfer_in_volume'] = df.groupby('customer_id')['bank_transfer_in_volume'].expanding().mean().reset_index(level=0, drop=True)
df['prior_mean_bank_transfer_out_volume'] = df.groupby('customer_id')['bank_transfer_out_volume'].expanding().mean().reset_index(level=0, drop=True)
df['prior_mean_crypto_in_volume'] = df.groupby('customer_id')['crypto_in_volume'].expanding().mean().reset_index(level=0, drop=True)
df['prior_mean_crypto_out_volume'] = df.groupby('customer_id')['crypto_out_volume'].expanding().mean().reset_index(level=0, drop=True)
df['prior_sum_bank_transfer_in_volume'] = df.groupby('customer_id')['bank_transfer_in_volume'].cumsum().values
df['prior_sum_bank_transfer_out_volume'] = df.groupby('customer_id')['bank_transfer_out_volume'].cumsum().values
df['prior_sum_crypto_in_volume'] = df.groupby('customer_id')['crypto_in_volume'].cumsum().values
df['prior_sum_crypto_out_volume'] = df.groupby('customer_id')['crypto_out_volume'].cumsum().values

df['prior_transfer_balance'] = (
    df.prior_sum_bank_transfer_in_volume - df.prior_sum_bank_transfer_out_volume
)

df['prior_crypto_balance'] = (
    df.prior_sum_crypto_in_volume - df.prior_sum_crypto_out_volume
)

df['prior_bank_balance'] = (
    (df.prior_sum_bank_transfer_in_volume - df.prior_sum_bank_transfer_out_volume)
    + (df.prior_sum_crypto_in_volume - df.prior_sum_crypto_out_volume)
)

df['prior_mean_balance'] = (
    (df.prior_mean_bank_transfer_in_volume - df.prior_mean_bank_transfer_out_volume)
    + (df.prior_mean_crypto_in_volume - df.prior_mean_crypto_out_volume)
)

Lifetime Window features...


### Processing categorical features

In [17]:
# Cast object cols to category
cat_features = ['job', 'country', 'broad_job_category']
for col in cat_features:
    df[col] = df[col].astype('str').fillna('')
    df[col] = df[col].astype('category')

### Defining diferent targets

In [18]:
# 365 days of inactivity
df = df.copy()
df['churn_365'] = 0
df.loc[df['days_diff'] >= 365, 'churn_365'] = 1

#420 days of inactivity
df['churn_420'] = 0
df.loc[df['days_diff'] >= 420, 'churn_420'] = 1

In [19]:
#All customers with a last activity date over a 18 months ago (reference date 2023-12-31, cutt-off date 2022-06-01).

'''
Verify this:

We're never very confident in our prediction at the point of churn.

We're asking a very hard question - is this transaction the last? Rather than asking has this customer churned after 100 days of inactivity have passed.

(Of the 2270378 in the train data, only 9820 are "final transactions", ~0.43%)
'''

churned_customers = df[df.date < '2024-01-01'].groupby('customer_id')['date'].max().reset_index()
churned_customers = churned_customers[churned_customers.date < '2022-06-01'].copy()
churned_customers.columns = ['customer_id', 'churn_date']
churned_customers['churn_18_months'] = 1

df = pd.merge(df, churned_customers, how='left', left_on=['customer_id', 'date'], right_on=['customer_id', 'churn_date'])
df.churn_18_months = df.churn_18_months.fillna(0)

In [20]:
df = reduce_mem_usage(df)

Memory usage of dataframe is 3791.33 MB
Memory usage after optimization is: 2767.88 MB
Decreased by 27.0%


### Saving the processed dataframe

In [21]:
# Cols that will not be used during the traning of the model
non_features_list = ['Id', 'customer_id','name','date_of_birth','address','date', 'job', 'touchpoints','csat_scores','Usage','churn','next_date','days_diff','split','churn_365','churn_420','churn_date','churn_18_months']

# Saving the features list
features_list = list(df.drop(columns=non_features_list).columns)
with open('./week_1/data/processed/features_list.json', 'w') as f:
    json.dump(features_list, f)

In [22]:
df.sort_values(by=['Id'], ascending=True, inplace=True)
df['Usage'] = df['Usage'].replace(0, 'Public')
df.to_parquet('./week_1/data/processed/feature_engineering_dataset_v2.parquet')